# CCUS Simulator

This simulator explores the trade-offs of **Carbon Capture, Utilization and Storage (CCUS)** with a simple but flexible Python model.  

The tool allows users to adjust assumptions and observe how costs, energy requirements, and climate outcomes change over time. It is not intended as a predictive model, but as an *explorable system* that highlights the main dynamics.

---

## Simulation modes
Three simulation approaches are available:

1. **Default simulation**  
   - A single global system with declining gross emissions and growing CCUS capacity.  
   - Capture cost and energy use per ton follow learning curves, improving with deployment.  
   - Provides a basic picture of “emissions vs. capture” with associated costs and energy use.

2. **Multi-sector simulation (simple ABM)**  
   - Emissions are divided into broad sectors: *Power, Industry, Transport, Buildings, Other*.  
   - Each sector has its own decline rate, while CCUS grows globally.  
   - Illustrates how sectoral differences affect total captured CO₂ and atmospheric concentrations.

3. **Agent-based simulation (ABM v2)**  
   - Individual agents (Power, Industry, Transport) respond to policies and shocks.  
   - Includes examples such as a recession year, a temporary CCUS setback, or stronger policies after a given date.  
   - Captures richer dynamics, such as faster growth once deployment passes a threshold, or short-term slowdowns.

---

## Controls
Parameters can be adjusted in the left panel:

- **Cost start (USD/tCO₂):** initial capture cost before learning.  
- **Learning rate (cost):** cost reduction each time cumulative capture doubles.  
- **Energy start (kWh/tCO₂):** initial electricity required to capture a ton of CO₂.  
- **Learning rate (energy):** efficiency gains in energy use over time.  
- **Grid CI (gCO₂/kWh):** carbon intensity of the grid supplying CCUS power.  
- **Power price (USD/MWh):** electricity cost for CCUS operations.  
- **Deployment scale (×):** scales the assumed CCUS rollout path up or down.  

Defaults can be restored at any time with the **Reset** button.

---

## Outputs
The right panel presents results in two sections:

- **Key indicators (last simulated year):**  
  - Captured CO₂ (Mt/yr)  
  - Levelized cost (USD/tCO₂)  
  - CCUS electricity demand (TWh/yr)  
  - Added CO₂ from CCUS energy use (Mt/yr)  
  - Atmospheric CO₂ (ppm)

- **Time series plots (with units):**  
  - Capture cost (USD/tCO₂)  
  - Energy intensity (kWh/tCO₂)  
  - Captured CO₂ (Mt/yr)  
  - CCUS electricity demand (TWh/yr)  
  - Atmospheric CO₂ (ppm)

---

## Model overview
1. Each year, gross emissions decline while CCUS deployment grows.  
2. Learning curves reduce capture cost and energy use with cumulative experience.  
3. Electricity demand from CCUS adds both financial cost and CO₂ emissions, depending on grid carbon intensity.  
4. Net emissions update atmospheric CO₂ via a simple carbon sink equation.  

The framework is intentionally simple so that assumptions remain transparent and the consequences of parameter choices are immediately visible.

---

## Notes
- This is not a forecast or detailed climate model.  
- Results are sensitive to input assumptions and should be interpreted as illustrations of possible dynamics.  
- The goal is to demonstrate how computational modelling can make abstract sustainability challenges more tangible.


In [16]:
#---------IMPORTS AND SETUP---------



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from ipywidgets import (
    FloatSlider, IntSlider, Dropdown, Label, GridBox, Layout,
    interactive_output, VBox
)

# ---------- helpers & data "cards" ----------
@dataclass
class CostEnergyKnobs:
    ccus_cost_usd_per_t0: float = 120.0   # starting $/t captured (tech-only)
    ccus_learning_rate_pct: float = 5.0   # cost ↓ %/yr
    ccus_energy_kWh_per_t0: float = 300.0 # kWh per t captured (start)
    ccus_energy_improve_pct: float = 2.0  # energy ↓ %/yr
    power_price_usd_per_kwh: float = 0.07 # $/kWh
    grid_kgCO2_per_kwh: float = 0.40      # kg CO2 / kWh

def _ppm_from_net_series(net_mt_series, starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2):
    ppm_vals, ppm_now = [], float(starting_ppm)
    for net_mt in np.asarray(net_mt_series, dtype=float):
        add_ppm = (net_mt / 1000.0) * ppm_per_GtCO2
        excess  = ppm_now - preindustrial_ppm
        sink    = k_sink * excess
        ppm_now = ppm_now + add_ppm - sink
        ppm_vals.append(ppm_now)
    return np.array(ppm_vals, dtype=float)

def _cost_energy_from_captured(captured_mt, knobs: CostEnergyKnobs,
                               cost0, learn_pct, kwh0, improve_pct, power_price, grid_kg_per_kwh):
    cap = np.asarray(captured_mt, dtype=float)
    n = len(cap)
    cost_per_t = cost0 * (1.0 - learn_pct/100.0) ** np.arange(n)
    kwh_per_t  = kwh0  * (1.0 - improve_pct/100.0) ** np.arange(n)

    tech_cost_usd_b   = cap * 1e6 * cost_per_t / 1e9      # convert Mt→tons, $→B$
    energy_twh        = cap * 1e6 * kwh_per_t / 1e12      # TWh
    energy_bill_usd_b = cap * 1e6 * kwh_per_t * power_price / 1e9
    energy_emis_mt    = cap * 1e6 * kwh_per_t * grid_kg_per_kwh / 1e9  # kg→Mt

    return (cost_per_t, kwh_per_t, tech_cost_usd_b, energy_twh, energy_bill_usd_b, energy_emis_mt)


# ---------- ABM v2: agents + policy/shock knobs ----------
from typing import Optional

@dataclass
class Agent:
    name: str
    gross_start_mt: float
    ccus_start_mt: float
    gross_decline_pct_pre: float
    gross_decline_pct_post: float
    ccus_growth_pct_pre: float
    ccus_growth_pct_post: float
    policy_year: int
    ccus_cap_mt: Optional[float] = None  # None = no cap

# Example agent set (you can change these later or expose as sliders)
AGENTS_V2 = [
    Agent("Power",     gross_start_mt=15000.0, ccus_start_mt=20.0,
          gross_decline_pct_pre=1.0,  gross_decline_pct_post=2.5,
          ccus_growth_pct_pre=18.0,   ccus_growth_pct_post=22.0,
          policy_year=2030, ccus_cap_mt=3000.0),
    Agent("Industry",  gross_start_mt=12000.0, ccus_start_mt=15.0,
          gross_decline_pct_pre=0.6,  gross_decline_pct_post=1.5,
          ccus_growth_pct_pre=10.0,   ccus_growth_pct_post=16.0,
          policy_year=2035, ccus_cap_mt=2500.0),
    Agent("Transport", gross_start_mt=8000.0,  ccus_start_mt=5.0,
          gross_decline_pct_pre=0.3,  gross_decline_pct_post=2.0,
          ccus_growth_pct_pre=8.0,    ccus_growth_pct_post=12.0,
          policy_year=2040, ccus_cap_mt=600.0),
]

# Global policy/shock/learning knobs (constants for now)
ABM2_LEARNING_THRESHOLD_MT = 5000.0   # when cumulative captured exceeds this, CCUS growth +2 pp
ABM2_RECESSION_YEAR        = 2032     # one-year macro shock on gross
ABM2_EXTRA_DECLINE         = 0.02     # extra -2% gross in recession year
ABM2_CCUS_SETBACK_YEAR     = 2043     # one-year CCUS setback
ABM2_CCUS_TEMP_CUT         = 0.10     # temporary -10% captured that year

    

In [17]:
# ---------- SIMULATORS ----------
def simulate_default_with_cost_energy(start_year, end_year,
                                      gross_start_mt, gross_decline_rate_pct,
                                      ccus_start_mt, ccus_growth_rate_pct,
                                      starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2,
                                      knobs: CostEnergyKnobs) -> pd.DataFrame:
    years = list(range(int(start_year), int(end_year)+1))
    g = float(gross_start_mt)
    c = float(ccus_start_mt)
    decline = 1.0 - float(gross_decline_rate_pct)/100.0
    growth  = 1.0 + float(ccus_growth_rate_pct)/100.0
    gross, captured = [], []

    
    for i, _ in enumerate(years):
        if i > 0:
            g *= decline
            c *= growth
        gross.append(g)
        captured.append(c)

        
    gross = np.array(gross, dtype=float)
    captured = np.array(captured, dtype=float)
    net = gross - captured

    ppm_vals = _ppm_from_net_series(net, starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2)

    (cost_per_t, kwh_per_t, tech_cost_b,
     energy_twh, energy_bill_b, energy_emis_mt) = _cost_energy_from_captured(     #calls the _cost_energy_from_captured function
        captured, knobs,
        knobs.ccus_cost_usd_per_t0, knobs.ccus_learning_rate_pct,
        knobs.ccus_energy_kWh_per_t0, knobs.ccus_energy_improve_pct,
        knobs.power_price_usd_per_kwh, knobs.grid_kgCO2_per_kwh
    )

    df = pd.DataFrame({
        "Year": years,
        "Gross_Mt": gross,
        "Captured_Mt": captured,
        "Net_Mt": net,
        "Atmospheric_CO2_ppm": ppm_vals,
        "CCUS_Cost_USD_per_t": cost_per_t,
        "CCUS_Energy_kWh_per_t": kwh_per_t,
        "CCUS_Tech_Cost_USD_B": tech_cost_b,
        "CCUS_Energy_TWh": energy_twh,
        "CCUS_Energy_Bill_USD_B": energy_bill_b,
        "Energy_Emissions_Mt": energy_emis_mt,
    })
    df["Effective_Net_Mt"] = df["Net_Mt"] + df["Energy_Emissions_Mt"]
    df["CCUS_Total_Spend_USD_B"] = df["CCUS_Tech_Cost_USD_B"] + df["CCUS_Energy_Bill_USD_B"]
    return df





    

In [18]:
#------- SIMPLE ABM -------------

ABM_SECTORS = ["Power","Industry","Transport","Buildings","Other"]

def simulate_abm_with_cost_energy(start_year, end_year,
                                  sector_gross_start_mt: dict, sector_decline_pct: dict,
                                  ccus_start_mt, ccus_growth_rate_pct,
                                  starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2,
                                  knobs: CostEnergyKnobs) -> pd.DataFrame:
    years = list(range(int(start_year), int(end_year)+1))
    current  = {s: float(sector_gross_start_mt[s]) for s in ABM_SECTORS}
    declines = {s: 1.0 - float(sector_decline_pct[s])/100.0 for s in ABM_SECTORS}
    c = float(ccus_start_mt)
    growth = 1.0 + float(ccus_growth_rate_pct)/100.0

    gross_list, captured_list, net_list = [], [], []
    for i, _ in enumerate(years):
        if i > 0:
            for s in ABM_SECTORS:
                current[s] *= declines[s]
            c *= growth
        g_total = sum(current.values())
        gross_list.append(g_total)
        captured_list.append(c)
        net_list.append(g_total - c)

    gross = np.array(gross_list, dtype=float)
    captured = np.array(captured_list, dtype=float)
    net = np.array(net_list, dtype=float)

    ppm_vals = _ppm_from_net_series(net, starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2)

    (cost_per_t, kwh_per_t, tech_cost_b,
     energy_twh, energy_bill_b, energy_emis_mt) = _cost_energy_from_captured(
        captured, knobs,
        knobs.ccus_cost_usd_per_t0, knobs.ccus_learning_rate_pct,
        knobs.ccus_energy_kWh_per_t0, knobs.ccus_energy_improve_pct,
        knobs.power_price_usd_per_kwh, knobs.grid_kgCO2_per_kwh
    )

    df = pd.DataFrame({
        "Year": years,
        "Gross_Mt": gross,
        "Captured_Mt": captured,
        "Net_Mt": net,
        "Atmospheric_CO2_ppm": ppm_vals,
        "CCUS_Cost_USD_per_t": cost_per_t,
        "CCUS_Energy_kWh_per_t": kwh_per_t,
        "CCUS_Tech_Cost_USD_B": tech_cost_b,
        "CCUS_Energy_TWh": energy_twh,
        "CCUS_Energy_Bill_USD_B": energy_bill_b,
        "Energy_Emissions_Mt": energy_emis_mt,
    })
    df["Effective_Net_Mt"] = df["Net_Mt"] + df["Energy_Emissions_Mt"]
    df["CCUS_Total_Spend_USD_B"] = df["CCUS_Tech_Cost_USD_B"] + df["CCUS_Energy_Bill_USD_B"]
    return df

In [19]:
# ---------- ABM + SHOCKS ----------


def simulate_abm_shocks_with_cost_energy(
    start_year, end_year,
    sector_gross_start_mt: dict,   # from sector sliders
    sector_decline_pct: dict,      # from sector sliders
    ccus_start_mt, ccus_growth_rate_pct,  # from CCUS sliders (global)
    starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2,
    knobs: CostEnergyKnobs,
    # shocks / learning (numbers; UI can pass them in)
    learning_threshold_mt=5000.0,        # cumulative captured threshold (Mt) to speed up CCUS growth
    recession_year=None,                 # e.g., 2032 (int) or None to disable
    extra_decline_this_year=0.0,         # fraction, e.g., 0.02 for -2% extra gross decline
    ccus_setback_year=None,              # e.g., 2043 (int) or None
    ccus_temporary_cut=0.0               # fraction, e.g., 0.10 for -10% captured that year
) -> pd.DataFrame:

    years = list(range(int(start_year), int(end_year) + 1))

    # initialize sector states and decline multipliers
    current  = {s: float(sector_gross_start_mt[s]) for s in ABM_SECTORS}
    declines = {s: 1.0 - float(sector_decline_pct[s]) / 100.0 for s in ABM_SECTORS}

    # CCUS (global) start + base growth factor
    c = float(ccus_start_mt)
    base_growth = 1.0 + float(ccus_growth_rate_pct) / 100.0

    gross_list, captured_list, net_list = [], [], []
    cumulative_captured = 0.0

    for i, y in enumerate(years):
        if i > 0:
            # update sectors with normal decline
            for s in ABM_SECTORS:
                current[s] *= declines[s]
                # one-year recession shock: apply extra decline this year only
                if recession_year is not None and int(y) == int(recession_year):
                    current[s] *= (1.0 - float(extra_decline_this_year))

            # learning bonus to CCUS growth once cumulative captured crosses threshold
            bonus_pp = 2.0 if cumulative_captured >= float(learning_threshold_mt) else 0.0
            growth_effective = 1.0 + (float(ccus_growth_rate_pct) + bonus_pp) / 100.0

            # update captured with effective growth
            c *= growth_effective

        # one-year CCUS setback applied after growth update
        if ccus_setback_year is not None and int(y) == int(ccus_setback_year):
            c *= (1.0 - float(ccus_temporary_cut))

        # totals
        g_total = sum(current.values())
        gross_list.append(g_total)
        captured_list.append(c)
        net_list.append(g_total - c)

        # update cumulative with actual captured this year (after any setback)
        cumulative_captured += c

    gross    = np.array(gross_list, dtype=float)
    captured = np.array(captured_list, dtype=float)
    net      = np.array(net_list, dtype=float)

    # ppm dynamics
    ppm_vals = _ppm_from_net_series(net, starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2)

    # costs/energy (NOTE: your _cost_energy_from_captured already has Mt→t, etc.)
    (cost_per_t, kwh_per_t, tech_cost_b,
     energy_twh, energy_bill_b, energy_emis_mt) = _cost_energy_from_captured(
        captured, knobs,
        knobs.ccus_cost_usd_per_t0, knobs.ccus_learning_rate_pct,
        knobs.ccus_energy_kWh_per_t0, knobs.ccus_energy_improve_pct,
        knobs.power_price_usd_per_kwh, knobs.grid_kgCO2_per_kwh
    )

    df = pd.DataFrame({
        "Year": years,
        "Gross_Mt": gross,
        "Captured_Mt": captured,
        "Net_Mt": net,
        "Atmospheric_CO2_ppm": ppm_vals,
        "CCUS_Cost_USD_per_t": cost_per_t,
        "CCUS_Energy_kWh_per_t": kwh_per_t,
        "CCUS_Tech_Cost_USD_B": tech_cost_b,
        "CCUS_Energy_TWh": energy_twh,
        "CCUS_Energy_Bill_USD_B": energy_bill_b,
        "Energy_Emissions_Mt": energy_emis_mt,
    })
    df["Effective_Net_Mt"] = df["Net_Mt"] + df["Energy_Emissions_Mt"]
    df["CCUS_Total_Spend_USD_B"] = df["CCUS_Tech_Cost_USD_B"] + df["CCUS_Energy_Bill_USD_B"]
    return df


In [20]:
# ---------- WIDGETS ----------
mode_dd = Dropdown(
    options=["Default scenario (live)", "Agent-based (live)", "ABM + shocks (live)"],
    value="Default scenario (live)"
)

start_year = IntSlider(min=2020, max=2030, step=1, value=2025)
end_year   = IntSlider(min=2030, max=2060, step=1, value=2050)

gross_start_mt         = IntSlider(min=10000, max=60000, step=500, value=35000)
gross_decline_rate_pct = FloatSlider(min=0.0, max=5.0, step=0.1, value=0.7)
ccus_start_mt          = IntSlider(min=0, max=500, step=5, value=40)
ccus_growth_rate_pct   = FloatSlider(min=0.0, max=50.0, step=0.5, value=15.0)

sector_start = {s: IntSlider(min=0, max=30000, step=250, value=v) for s, v in
                zip(ABM_SECTORS, [12000, 9000, 7000, 5000, 2000])}
sector_decl  = {s: FloatSlider(min=0.0, max=10.0, step=0.1, value=v) for s, v in
                zip(ABM_SECTORS, [2.5, 1.5, 1.0, 1.8, 0.5])}

starting_ppm        = FloatSlider(min=350, max=500, step=1, value=420)
preindustrial_ppm   = FloatSlider(min=250, max=300, step=0.5, value=280)
k_sink              = FloatSlider(min=0.0, max=0.05, step=0.001, value=0.012)
ppm_per_GtCO2       = FloatSlider(min=0.08, max=0.20, step=0.001, value=1.0/7.8)

ccus_cost_usd_per_t0   = FloatSlider(min=20,  max=600,  step=5,   value=120)
ccus_learning_rate_pct = FloatSlider(min=0,   max=30,   step=0.5, value=5.0)
ccus_energy_kwh_per_t0 = FloatSlider(min=50,  max=1200, step=10,  value=300)
ccus_energy_improve_pct= FloatSlider(min=0,   max=20,   step=0.5, value=2.0)
power_price_usd_per_kwh= FloatSlider(min=0.02,max=0.3,  step=0.005,value=0.07)
grid_kgCO2_per_kwh     = FloatSlider(min=0.0, max=0.9,  step=0.01, value=0.40)

# --- SHOCKS & LEARNING (new) ---
learning_threshold_mt = FloatSlider(min=0, max=50000, step=100, value=5000.0)
recession_year        = IntSlider(min=2020, max=2060, step=1, value=2032)
extra_decline_pct     = FloatSlider(min=0.0, max=10.0, step=0.1, value=2.0)   # % (we'll convert to fraction)
ccus_setback_year     = IntSlider(min=2020, max=2060, step=1, value=2043)
ccus_cut_pct          = FloatSlider(min=0.0, max=50.0, step=0.5, value=10.0)  # % (we'll convert to fraction)

from ipywidgets import Label, GridBox, Layout, VBox, interactive_output

def _grid_rows(rows):
    children = []
    for label, widget in rows:
        children += [Label(label), widget]
    return GridBox(children=children,
                   layout=Layout(grid_template_columns="220px 520px", grid_gap="6px 12px", width="800px"))

grid_mode = _grid_rows([("Mode:", mode_dd)])

grid_default = _grid_rows([
    ("Start year:", start_year), ("End year:", end_year),
    ("Gross start (Mt):", gross_start_mt),
    ("Gross decline %/yr:", gross_decline_rate_pct),
    ("CCUS start (Mt):", ccus_start_mt),
    ("CCUS growth %/yr:", ccus_growth_rate_pct),
])

grid_abm = _grid_rows(
    [("Start year:", start_year), ("End year:", end_year)] +
    [(f"{s} start (Mt):", sector_start[s]) for s in ABM_SECTORS] +
    [(f"{s} decline %/yr:", sector_decl[s]) for s in ABM_SECTORS] +
    [("CCUS start (Mt):", ccus_start_mt), ("CCUS growth %/yr:", ccus_growth_rate_pct)]
)

grid_ppm = _grid_rows([
    ("Start ppm:", starting_ppm),
    ("Preindustrial ppm:", preindustrial_ppm),
    ("Sink rate /yr:", k_sink),
    ("ppm per GtCO₂:", ppm_per_GtCO2),
])

grid_cost = _grid_rows([
    ("CCUS $/t (start):", ccus_cost_usd_per_t0),
    ("Learning %/yr:", ccus_learning_rate_pct),
    ("Energy kWh/t (start):", ccus_energy_kwh_per_t0),
    ("Energy improve %/yr:", ccus_energy_improve_pct),
    ("Power price $/kWh:", power_price_usd_per_kwh),
    ("Grid kgCO₂/kWh:", grid_kgCO2_per_kwh),
])

grid_shocks = _grid_rows([
    ("Learning threshold (Mt):", learning_threshold_mt),
    ("Recession year:", recession_year),
    ("Extra gross decline in shock year (%):", extra_decline_pct),
    ("CCUS setback year:", ccus_setback_year),
    ("CCUS temporary cut (%):", ccus_cut_pct),
])

def _toggle_grids(*_):
    if mode_dd.value == "Default scenario (live)":
        grid_default.layout.display = ""
        grid_abm.layout.display = "none"
        grid_shocks.layout.display = "none"
    elif mode_dd.value == "Agent-based (live)":
        grid_default.layout.display = "none"
        grid_abm.layout.display = ""
        grid_shocks.layout.display = "none"
    else:  # "ABM + shocks (live)"
        grid_default.layout.display = "none"
        grid_abm.layout.display = ""
        grid_shocks.layout.display = ""
mode_dd.observe(_toggle_grids, names="value")
_toggle_grids()

def _read_sector_widgets():
    sector_gross = {s: float(sector_start[s].value) for s in ABM_SECTORS}
    sector_declp = {s: float(sector_decl[s].value)  for s in ABM_SECTORS}
    return sector_gross, sector_declp

def _ui(mode,
        gross_start_mt, gross_decline_rate_pct,
        ccus_start_mt, ccus_growth_rate_pct,
        start_year, end_year,
        starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2,
        ccus_cost_usd_per_t0, ccus_learning_rate_pct,
        ccus_energy_kwh_per_t0, ccus_energy_improve_pct,
        power_price_usd_per_kwh, grid_kgCO2_per_kwh,
        # shocks/learning inputs from UI:
        learning_threshold_mt, recession_year, extra_decline_pct, ccus_setback_year, ccus_cut_pct):

    knobs = CostEnergyKnobs(
        ccus_cost_usd_per_t0=float(ccus_cost_usd_per_t0),
        ccus_learning_rate_pct=float(ccus_learning_rate_pct),
        ccus_energy_kWh_per_t0=float(ccus_energy_kwh_per_t0),
        ccus_energy_improve_pct=float(ccus_energy_improve_pct),
        power_price_usd_per_kwh=float(power_price_usd_per_kwh),
        grid_kgCO2_per_kwh=float(grid_kgCO2_per_kwh),
    )

    if mode == "Default scenario (live)":
        df = simulate_default_with_cost_energy(
            start_year, end_year,
            gross_start_mt, gross_decline_rate_pct,
            ccus_start_mt, ccus_growth_rate_pct,
            starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2,
            knobs
        )
        used = "default scenario (live)"

    elif mode == "Agent-based (live)":
        sector_gross, sector_declp = _read_sector_widgets()
        df = simulate_abm_with_cost_energy(
            start_year, end_year,
            sector_gross, sector_declp,
            ccus_start_mt, ccus_growth_rate_pct,
            starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2,
            knobs
        )
        used = "agent-based (live)"

    else:  # "ABM + shocks (live)" -> uses sector sliders + shock sliders
        sector_gross, sector_declp = _read_sector_widgets()
        df = simulate_abm_shocks_with_cost_energy(
            start_year, end_year,
            sector_gross, sector_declp,
            ccus_start_mt, ccus_growth_rate_pct,
            starting_ppm, preindustrial_ppm, k_sink, ppm_per_GtCO2,
            knobs,
            learning_threshold_mt=float(learning_threshold_mt),
            recession_year=int(recession_year),
            extra_decline_this_year=float(extra_decline_pct) / 100.0,  # % → fraction
            ccus_setback_year=int(ccus_setback_year),
            ccus_temporary_cut=float(ccus_cut_pct) / 100.0             # % → fraction
        )
        used = "agent-based + shocks (live)"

    # --- plot (with fixed legend) ---
    plt.figure(figsize=(9,5))
    ax1 = plt.gca()
    ax2 = ax1.twinx()

    ax1.plot(df["Year"], df["Gross_Mt"],    label="Gross (Mt)")
    ax1.plot(df["Year"], df["Captured_Mt"], label="Captured (Mt)")
    ax1.plot(df["Year"], df["Net_Mt"],      label="Net (Mt)")
    ax1.plot(df["Year"], df["Effective_Net_Mt"], label="Effective Net (Mt)", linestyle="--")

    ax1.set_xlabel("Year"); ax1.set_ylabel("Emissions (Mt/yr)"); ax1.grid(True)
    ax2.plot(df["Year"], df["Atmospheric_CO2_ppm"], color="purple", label="CO₂ ppm")
    ax2.set_ylabel("CO₂ (ppm)")

    h1, lab1 = ax1.get_legend_handles_labels()
    h2, lab2 = ax2.get_legend_handles_labels()
    ax1.legend(h1 + h2, lab1 + lab2, loc="upper right")

    plt.title(f"Emissions, Effective Net & CO₂ (ppm) — {used}")
    plt.show()

    last = df.iloc[-1]
    print(
        f"Final {int(last['Year'])}: "
        f"Captured={last['Captured_Mt']:,.0f} Mt | "
        f"CCUS $/t={last['CCUS_Cost_USD_per_t']:,.0f} | "
        f"Tech cost=${last['CCUS_Tech_Cost_USD_B']:,.2f} B | "
        f"Energy={last['CCUS_Energy_TWh']:,.1f} TWh | "
        f"Energy bill=${last['CCUS_Energy_Bill_USD_B']:,.2f} B | "
        f"Energy emis={last['Energy_Emissions_Mt']:,.1f} Mt | "
        f"Total spend=${last['CCUS_Total_Spend_USD_B']:,.2f} B | "
        f"ppm={last['Atmospheric_CO2_ppm']:,.1f}"
    )

# wire sliders -> UI and display
out = interactive_output(
    _ui,
    {
        "mode": mode_dd,
        "gross_start_mt": gross_start_mt,
        "gross_decline_rate_pct": gross_decline_rate_pct,
        "ccus_start_mt": ccus_start_mt,
        "ccus_growth_rate_pct": ccus_growth_rate_pct,
        "start_year": start_year,
        "end_year": end_year,
        "starting_ppm": starting_ppm,
        "preindustrial_ppm": preindustrial_ppm,
        "k_sink": k_sink,
        "ppm_per_GtCO2": ppm_per_GtCO2,
        "ccus_cost_usd_per_t0": ccus_cost_usd_per_t0,
        "ccus_learning_rate_pct": ccus_learning_rate_pct,
        "ccus_energy_kwh_per_t0": ccus_energy_kwh_per_t0,
        "ccus_energy_improve_pct": ccus_energy_improve_pct,
        "power_price_usd_per_kwh": power_price_usd_per_kwh,
        "grid_kgCO2_per_kwh": grid_kgCO2_per_kwh,
        # shocks/learning wires:
        "learning_threshold_mt": learning_threshold_mt,
        "recession_year": recession_year,
        "extra_decline_pct": extra_decline_pct,
        "ccus_setback_year": ccus_setback_year,
        "ccus_cut_pct": ccus_cut_pct,
    }
)

# assemble UI
ui = VBox([grid_mode, grid_default, grid_abm, grid_shocks, grid_ppm, grid_cost, out])
display(ui)
